# Pre-processing wikipedia dumps with dask (part 1)
> Wikipedia SQL dumps are a great data source, of manageable size (when compared to the full dumps). However, processing them efficiently can be challenging. Take advantage of the (hopefully) many CPU cores available to process them. Part 1 shows how to convert the SQL dumps into dataframes and save them as parquet files.

- toc: true
- branch: master
- badges: true
- comments: false
- author: Nicolas Aspert
- categories: [wikipedia, dask]

# Introduction

In addition to being a great source of information, [Wikipedia](https://www.wikipedia.org) is also a model in openness and access to its data. You can access the whole data from the [dumps page](https://dumps.wikimedia.org/) and download most of the data powering Wikipedia. 

I will focus here on how to process the SQL dumps and transform them to be ingested by, for instance, a [Neo4j graph database](https://neo4j.com). This can be done by other means, for instance using the [sparkwiki](https://github.com/epfl-lts2/sparkwiki) tool (disclaimer: mostly written by me). Sparkwiki is great, however it requires a [spark](https://spark.apache.org/) instance/cluster to work, which is not always easy to set up, even with [Bigtop](https://blog.miz.space/tutorial/2019/04/04/how-to-install-spark-using-apache-bigtop-ubuntu/) or [elasticluster](https://elasticluster.readthedocs.io/en/latest/playbooks.html#hadoop-spark). Nowadays, the research world runs a lot of Python, and [dask](https://dask.org) seems (claims ?) to be the Python-based Spark equivalent, so making an equivalent of sparkwiki in python is a good excuse to see if dask is fit for the job. 

## Requirements

In order to run the experiments below, you need
- a good internet connection (the dumps amount for ca. 8 GB of data you need to download)
- a computer with a decent amount of RAM and CPUs (I ran them on a 2x20-cores system with 128 GB of RAM, but never used it fully) or a suitable cluster if you have one at hand, and preferrably SSD local storage. I will asssume a Linux-based OS, though it should work on MacOS or even Windows.
- a suitable conda/pip environment with the necessary packages installed (TODO)


## SQL dumps

In this example, I will show how to process the english wikipedia database dump, the adaptation to other languages should be fairly straightforward. You can find the index of all dumps on [this page](https://dumps.wikimedia.org/backup-index.html), the english language SQL dumps are in the [enwiki subfolder](https://dumps.wikimedia.org/enwiki/). In order to achieve speedy downloads, you can use of the [mirrors close to you](https://dumps.wikimedia.org/mirrors.html) (I am in Switzerland, the mirror from Umeå university is the fastest one for me, YMMV). The latest backup available (when I wrote this) is from Nov. 20th 2020, I will be using this one in my examples.

### Files needed

You will need 3 files :
- [enwiki-20201120-page.sql.gz](https://dumps.wikimedia.org/enwiki/20201120/enwiki-20201120-page.sql.gz) which contains page data (title, namespace, etc.)
- [enwiki-20201120-pagelinks.sql.gz](https://dumps.wikimedia.org/enwiki/20201120/enwiki-20201120-pagelinks.sql.gz) which contains all links between pages within the English wikipedia edition (links across languages are stored in a different table)
- [enwiki-20201120-redirect.sql.gz](https://dumps.wikimedia.org/enwiki/20201120/enwiki-20201120-redirect.sql.gz) which contains redirection information betweem pages (e.g when a page is renamed)

You can find more detailed information about each table on the [database layout](https://www.mediawiki.org/wiki/Manual:Database_layout) page.

In [ ]:
# directory where data is stored - adjust to your needs
data_path = '/data/wikipedia/20201120'

### Parallelism - preliminary step

Sparkwiki was able to process the compressed dumps in parallel after converting them to bz2 archives. After experimenting with dask, it turns out this simple step is not sufficient to achieve parallelism. However, dask supports [reading multiple files in parallel](https://docs.dask.org/en/latest/bag-api.html#dask.bag.read_text), so an extra step is needed to convert the single-file SQL dump into multiple files. Those files can be used to re-create the database powering Wikipedia, and is just a big text file containing `INSERT` statements (quite a lot of them actually). 

The handy Linux command `split` allows cutting the files into smaller chunks, without breaking SQL statements which need to be parsed later on:
```
zcat enwiki-20201101-redirect.sql.gz | split -l 100 --filter 'bzip2 > $FILE.bz2' - split-redirect-
```

This command will cut the redirect SQL dump into 100-lines bzip2-compressed (thanks to the `filter` parameter) chunks with a `split-redirect-` prefix. Check the [man page](https://man7.org/linux/man-pages/man1/split.1.html) of the command for more details.

We can now split the two other files. The number of lines in each chunk is hand-adjusted to have a good compromise between number of files and file size :
```
zcat enwiki-20201101-page.sql.gz | split -l 150 --filter 'bzip2 > $FILE.bz2' - split-page-
zcat enwiki-20201101-pagelinks.sql.gz | split -l 400 --filter 'bzip2 > $FILE.bz2' - split-pagelinks-
```

**This can take a few hours** (esp. the splitting of the pagelinks file), so if you want to skip this part you can use the splits I uploaded on our S3-compatible storage hosted on SwitchEngines.

# Processing SQL dumps 

We are now ready to actually perform the processing. 

## Imports 

In [42]:
import pandas as pd
import dask
import dask.bag as db
import dask.dataframe as ddf
import re
import os

## Starting the dask (local) cluster

Before running commands, we need to set up a few things for processing. NB: This is NOT the only way to do it, you can read about setting up the scheduler on a [single machine](https://docs.dask.org/en/latest/setup/single-distributed.html), or on [other types of environments](https://docs.dask.org/en/latest/setup.html) such as clouds, Kubernetes, etc.
One important thing to set up when running on a single computer is to have a **local temporary directory**: at some point when processing, dask will write data to disk. It is fairly common to have NFS-mounted directories for instance, and it is crucial for good performance to make sure data does not need to go through a network.

In [ ]:
dask.config.set({'temporary_directory': '/tmp'}) # make sure temp dir is local and has sufficient space. Adjust to your resources/needs !

In [4]:
from dask.distributed import LocalCluster, Client

We can now start our local cluster. This will work for me, make sure you adapt to your local resources ! It is important to keep $\text{n_workers} \times \text{memory_limit}$ under the physical memory available (or a lower value if the computer is shared with other users and you want to be nice to others).

In [5]:
cluster = LocalCluster(n_workers=6, threads_per_worker=4, memory_limit=24e9)
client = Client(cluster)

## Helpers functions

Let us now set up a few routines and regexps (check [sparkwiki](https://github.com/epfl-lts2/sparkwiki) for details)

In [6]:
re_dict = {"page": "\((\d+),(\d+),'(.*?)','(.*?)',([01]),([01]),([\d\.]+?),'(\d{14})',(.*?),(\d+),(\d+),(.*?),(.*?)\)",
          "redirect":"\((\d+),(\d+),'(.*?)',(.*?),(.*?)\)",
          "pagelinks":"\((\d+),(\d+),'(.*?)',(\d+)\)"}

In [7]:
# dismiss additional statements (CREATE TABLE, etc.) from the file
def filter_line(line, dump_type):
    return line.startswith('INSERT INTO `{}` VALUES'.format(dump_type))

In [8]:
def split_line(line):
    return line.split(" VALUES ")[1].strip()

In [9]:
def get_redirect(rec):
    return {'from':int(rec[0]), 'target_ns':int(rec[1]), 'title':rec[2], 'inter_wiki':rec[3], 'fragment':rec[4]}

In [10]:
def get_page(rec):
    # case class WikipediaSimplePage(id:Int, title:String, isRedirect:Boolean, isNew: Boolean)
    return {'id': int(rec[0]), 'namespace':int(rec[1]), 'title': rec[2], 'is_redirect':int(rec[4])==1, 'is_new':int(rec[5])==1}

In [11]:
def get_pagelinks(rec):
    # case class WikipediaPagelinks(from:Int, namespace:Int, title:String, fromNamespace:Int)
    return {'from': int(rec[0]), 'namespace':int(rec[1]), 'title':rec[2], 'from_namespace':int(rec[3])}

# Processing redirects

It may seem surprising to start with this particular table. As it is the smallest one, it is often more convenient to start and experiment processing with this one. If you make a mistake, you don't have to wait for a long time before the crash.

## Read the local files into a dask bag

A [dask bag](https://docs.dask.org/en/latest/bag.html) is the equivalent of a [Spark RDD](https://spark.apache.org/docs/latest/rdd-programming-guide.html#resilient-distributed-datasets-rdds). It supports simple operations such as `filter`, `map`, etc.

In [ ]:
redirects_bag = db.read_text(os.path.join(data_path, 'splits/split-redirect-*.bz2')

## Alternative: read the files from S3 bucket

It is not hosted on Amazon but on the Swiss Universities cloud [SwitchEngines](https://www.switch.ch/engines/), hence the custom options needed below

In [12]:
storage_options={'anon':True, 'client_kwargs':{'endpoint_url':'https://os.unil.cloud.switch.ch'}}

In [13]:
redirects_bag = db.read_text('s3://lts2-wikipedia/enwiki/20201120/splits/split-redirect-*.bz2', storage_options=storage_options)

## Tansform each record into a redirect dictionary

We can chain conveniently the operators to 
- filter out non INSERT statements using `filter_line`
- split all VALUES from the INSERT using `split_line` and the appropriate regexp
- convert to a dictionary using `get_redirect`

Sparkwiki does the same operations, check the [WikipediaElementParser.scala](https://github.com/epfl-lts2/sparkwiki/blob/master/src/main/scala/ch/epfl/lts2/wikipedia/WikipediaElementParser.scala) file for more details.

In [44]:
redirects = redirects_bag.filter(lambda x: filter_line(x, 'redirect'))\
                         .map(split_line)\
                         .map(lambda x:re.findall(re_dict['redirect'], x)).flatten()\
                         .map(get_redirect)

Finally, the collection of small redirect dictionaries is converted to a dask DataFrame (similar to a pandas DataFrame)

In [45]:
redirects_df = redirects.to_dataframe()

Filter out all redirects that do not concern namespace 0 (= articles), cf. [Wikipedia namespaces](https://en.wikipedia.org/wiki/Wikipedia:Namespace)

In [24]:
redirects_df_filt = redirects_df[redirects_df['target_ns']==0]

At this point, nothing is computed yet. Setting the index and saving the resulting DataFrame to a parquet file will trigger all computations. You can monitor what is happening under the hood by opening a connection to the [dask scheduler web interface](https://distributed.dask.org/en/1.9.5/web.html) which should run on port 8787. It takes less than a minute on the system I have at hand.

In [25]:
redirects_df_filt.set_index('from').to_parquet(os.path.join(data_path, 'processed', 'redirect.parquet'), compression='gzip', engine='fastparquet')

You can call the usual pandas DataFrame methos such as `head`, `tail`etc. However, calling them directly on `redirects_df_filt`will trigger a computation. If you want to avoid this, you can reload it from disk :

In [43]:
# optional
redirects_df_reloaded = ddf.read_parquet(os.path.join(data_path, 'processed', 'redirect.parquet'))
redirects_df_reloaded.head()

,target_ns,title,inter_wiki,fragment
from,,,,
10,0,Computer_accessibility,'',''
13,0,History_of_Afghanistan,'',''
14,0,Geography_of_Afghanistan,'',''
15,0,Demographics_of_Afghanistan,'',''
18,0,Communications_in_Afghanistan,'',''


# Processing pages

Almost identical to redirects, takes longer though

## Read splits from local filesystem

In [ ]:
pages_bag = db.read_text(os.path.join(data_path, 'splits/split-page-*.bz2')

## Alternative: read from S3 storage

In [26]:
pages_bag = db.read_text('s3://lts2-wikipedia/enwiki/20201120/splits/split-page-*.bz2', storage_options=storage_options)

## Filter records and create dataframe

In [28]:
pages = pages_bag.filter(lambda x: filter_line(x, 'page'))\
                 .map(split_line)\
                 .map(lambda x:re.findall(re_dict['page'], x))\
                 .flatten().map(get_page)

In [29]:
pages_df = pages.to_dataframe()

Keep only namespace 0 pages

In [30]:
pages_filt_df = pages_df[pages_df['namespace']==0]

Trigger computation and save result (runs for a few minutes on my system)

In [31]:
pages_filt_df.set_index('id').to_parquet(os.path.join(data_path, 'processed', 'pages.parquet'), compression='gzip', engine='fastparquet')

Check what we now have in the dataframe : 

In [52]:
# optional reload from disk
pages_df_reloaded = ddf.read_parquet(os.path.join(data_path, 'processed', 'pages.parquet'))
pages_df_reloaded.head(20)

,namespace,title,is_redirect,is_new
id,,,,
10,0,AccessibleComputing,True,False
12,0,Anarchism,False,False
13,0,AfghanistanHistory,True,False
14,0,AfghanistanGeography,True,False
15,0,AfghanistanPeople,True,False
18,0,AfghanistanCommunications,True,False
19,0,AfghanistanTransportations,True,False
20,0,AfghanistanMilitary,True,False
21,0,AfghanistanTransnationalIssues,True,False


# Processing pagelinks

NB: this is the biggest part to process. It seems some entries can trigger utf-8 decoding errors, hence the `errors='backslashreplace'` addition whem reading.
No index will be created to speed up computation

In [32]:
pagelinks_bag = db.read_text(os.path.join(data_path, 'splits/split-pagelinks-*.bz2'), errors='backslashreplace')

Or S3-hosted splits :

In [ ]:
pageslinks_bag = db.read_text('s3://lts2-wikipedia/enwiki/20201120/splits/split-pagelinks-*.bz2', storage_options=storage_options, errors='backslashreplace')

get pagelinks dictionaries

In [33]:
pagelinks = pagelinks_bag.filter(lambda x: filter_line(x, 'pagelinks'))\
                         .map(split_line)\
                         .map(lambda x:re.findall(re_dict['pagelinks'], x))\
                         .flatten().map(get_pagelinks)

In [34]:
pagelinks_df = pagelinks.to_dataframe()

Only keep links between articles (namespace == 0), discard all the others

In [35]:
pagelinks_filt_df = pagelinks_df[(pagelinks_df['namespace'] == 0) & (pagelinks_df['from_namespace'] == 0)]

In [40]:
pagelinks_filt_df.to_parquet(os.path.join(data_path, 'processed', 'pagelinks.parquet'), compression='gzip', engine='fastparquet')

In [49]:
#optional reload from disk
pagelinks_reloaded = ddf.read_parquet(os.path.join(data_path, 'processed', 'pagelinks.parquet'))
pagelinks_reloaded.head(20)

,from,namespace,title,from_namespace
0,4748,0,!,0
1,9773,0,!,0
2,15154,0,!,0
3,25213,0,!,0
4,613303,0,!,0
5,1028188,0,!,0
6,1497620,0,!,0
7,2875276,0,!,0
8,2988645,0,!,0
9,4355567,0,!,0
